In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pickle

# retrieve X_data, y_data, X_test, y_
X_data = pickle.load(open('/content/gdrive/MyDrive/svm content classification/X_data.pkl', 'rb'))
y_data = pickle.load(open('/content/gdrive/MyDrive/svm content classification/y_data.pkl', 'rb'))
X_test = pickle.load(open('/content/gdrive/MyDrive/svm content classification/X_test.pkl', 'rb'))
y_test = pickle.load(open('/content/gdrive/MyDrive/svm content classification/y_test.pkl', 'rb'))

In [ ]:
stopwords = []

# open file .txt contains vietnamese stopwords
with open ('/content/gdrive/MyDrive/svm content classification/vietnamese-stopwords.txt', 'r', encoding='utf-8') as f:
  while True:
    word = f.readline()
    if not word: break
    stopwords.append(word[:-1])

print(stopwords)

['bị', 'bởi', 'cả', 'các', 'cái', 'cần', 'càng', 'chỉ', 'chiếc', 'cho', 'chứ', 'chưa', 'chuyện', 'có', 'có thể', 'cứ', 'của', 'cùng', 'cũng', 'đã', 'đang', 'đây', 'để', 'đến nỗi', 'đều', 'điều', 'do', 'đó', 'được', 'dưới', 'gì', 'khi', 'không', 'là', 'lại', 'lên', 'lúc', 'mà', 'mỗi', 'một cách', 'này', 'nên', 'nếu', 'ngay', 'nhiều', 'như', 'nhưng', 'những', 'nơi', 'nữa', 'phải', 'qua', 'ra', 'rằng', 'rất', 'rồi', 'sau', 'sẽ', 'so', 'sự', 'tại', 'theo', 'thì', 'trên', 'trước', 'từ', 'từng', 'và', 'vẫn', 'vào', 'vậy', 'vì', 'việc', 'với', 'vừa', 'nhận', 'cao', 'nhà', 'quá', 'riêng', 'muốn', 'số', 'thấy', 'hay', 'lần', 'nào', 'bằng', 'biết', 'lớn', 'khác', 'thời gian', 'họ', 'tháng', 'chính', 'nói', 'đi', 'tới', 'tôi', 'làm', 'mới', 'ngày', 'mình', 'còn', 'năm', 'nhất', 'hơn', 'ông', 'anh', 'đến', 'người', 'ở', 'về', 'một', 'trong', 'thực sự', 'ở trên', 'tất cả', 'hầu hết', 'luôn', 'giữa', 'bất kỳ', 'hỏi', 'bạn', 'cô', 'tớ', 'cậu', 'bác', 'chú', 'dì', 'thím', 'mợ', 'bà', 'em', 'thường', '

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# apply TF-IDF technique into vectorizing words
tfidf_vect = TfidfVectorizer(analyzer='word', stop_words=stopwords)

# apply into training data to create vocabulary 
X_train_tfidf = tfidf_vect.fit_transform(X_data)

# TF-IDF X_test
X_test_tfidf =  tfidf_vect.transform(X_test)

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=500, random_state=17)

svd.fit(X_train_tfidf)
X_data_tfidf_svd = svd.transform(X_train_tfidf)
X_test_tfidf_svd = svd.transform(X_test_tfidf)

In [ ]:
from sklearn import preprocessing

encoder = preprocessing.LabelEncoder()
y_data = encoder.fit_transform(y_data)
y_test = encoder.fit_transform(y_test)
print(y_data)

[26 26 26 ...  0  0  0]


In [ ]:
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras import models
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn import metrics

def create_dnn_model():
  input_layer = Input(shape=(500,))
  layer = Dense(1024, activation='relu')(input_layer)
  layer = Dense(1024, activation='relu')(layer)
  layer = Dense(512, activation='relu')(layer)
  layer = Dense(256, activation='relu')(layer)
  layer = Dense(128, activation='relu')(layer)
  layer = Dense(64, activation='relu')(layer)
  layer = Dense(32, activation='relu')(layer)
  output_layer = Dense(27, activation='softmax')(layer)
    
  classifier = models.Model(input_layer, output_layer)
  classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  
  classifier.summary()

  return classifier

def train_model(classifier, X_data, y_data, X_test, y_test, is_neuralnet=False, n_epochs=100):       
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, random_state=42)
    
    if is_neuralnet:
        classifier.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, batch_size=512, verbose=0)
        
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        val_predictions = val_predictions.argmax(axis=-1)
        test_predictions = test_predictions.argmax(axis=-1)
    else:
        classifier.fit(X_train, y_train)
    
        train_predictions = classifier.predict(X_train)
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        
    print("Validation accuracy: ", metrics.accuracy_score(val_predictions, y_val))
    print("Test accuracy: ", metrics.accuracy_score(test_predictions, y_test))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional, Reshape

def create_lstm_model():
    input_layer = Input(shape=(500,))
    
    layer = Reshape((10, 50))(input_layer)
    layer = LSTM(128, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(256, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)
    layer = Dense(64, activation='relu')(layer)
    layer = Dense(32, activation='relu')(layer)
    
    output_layer = Dense(27, activation='softmax')(layer)
    
    classifier = models.Model(input_layer, output_layer)
    
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    classifier.summary()

    return classifier

In [ ]:
classifier = create_dnn_model()
train_model(classifier=classifier, X_data=X_data_tfidf_svd, y_data=y_data, X_test=X_test_tfidf_svd, y_test=y_test, is_neuralnet=True)

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 500)]             0         
_________________________________________________________________
dense_47 (Dense)             (None, 1024)              513024    
_________________________________________________________________
dense_48 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_49 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_50 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_51 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_52 (Dense)             (None, 64)                8256

In [ ]:
classifier = create_lstm_model()
train_model(classifier=classifier, X_data=X_data_tfidf_svd, y_data=y_data, X_test=X_test_tfidf_svd, y_test=y_test, is_neuralnet=True)

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 500)]             0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 10, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dense_55 (Dense)             (None, 512)               66048     
_________________________________________________________________
dense_56 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_57 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_58 (Dense)             (None, 128)               3289

In [ ]:
from sklearn.decomposition import TruncatedSVD

pca = TruncatedSVD(n_components=500, random_state=17)
X_train_pca = pca.fit_transform(X_train_tfidf)
X_test_pca = pca.transform(X_test_tfidf)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

def train_model(classifier, X_data, y_data):       
  X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, random_state=42)
    
  classifier.fit(X_train, y_train)
    
  # train_predictions = classifier.predict(X_train)
  val_predictions = classifier.predict(X_val)
        
  print("Validation accuracy: ", metrics.accuracy_score(val_predictions, y_val))


def test_model(classifier, X_test, y_test):

  test_predictions = classifier.predict(X_test)
  print("Test accuracy: ", metrics.accuracy_score(test_predictions, y_test))
  print(metrics.classification_report(y_test, test_predictions, digits=27))

  return test_predictions

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn import svm

pca = TruncatedSVD(n_components=1000, random_state=17)
X_train_pca = pca.fit_transform(X_train_tfidf)
X_test_pca = pca.transform(X_test_tfidf)

svm_model = svm.SVC()
train_model(svm_model, X_train_pca, y_data)

Validation accuracy:  0.9054242002781642


In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn import svm

svm_model = svm.SVC()

n_components = [300, 400, 500, 600, 700]

for n_component in n_components:
  print("Training with", n_component, "components: ")
  pca = TruncatedSVD(n_components=n_component, random_state=17)
  X_train_pca = pca.fit_transform(X_train_tfidf)
  X_test_pca = pca.transform(X_test_tfidf)
  train_model(svm_model, X_train_pca, y_train)

Training with 300 components: 
Validation accuracy:  0.8991655076495132
Training with 400 components: 
Validation accuracy:  0.9019471488178025
Training with 500 components: 
Validation accuracy:  0.9054242002781642
Training with 600 components: 
Validation accuracy:  0.9040333796940194
Training with 700 components: 
Validation accuracy:  0.9061196105702365


In [ ]:
# test model with pca 700 principal components
test_model(svm_model, X_test_pca, y_test)

Test accuracy:  0.9115601192447831
                             precision    recall  f1-score   support

                          0  0.930864197530864245777593169 0.927429274292742911178777376 0.929143561306223109141910754       813
                          1  0.981723237597911246865578505 0.939999999999999946709294818 0.960408684546615543098369017       400
                          2  0.981884057971014523324981838 0.960992907801418438928919841 0.971326164874552033801080597       282
                          3  0.997258396161754601116911090 0.993852459016393408042233659 0.995552514539856270303630481      1464
                          4  0.983974358974358920271185980 0.959374999999999977795539507 0.971518987341772111143711754       320
                          5  0.986876640419947492866015182 0.986876640419947492866015182 0.986876640419947492866015182       381
                          6  0.645390070921985858909408762 0.898765432098765471025103579 0.751289989680082648249026533   

array([19, 19, 15, ...,  1,  1,  1])